In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import open3d as o3d
import os
from PIL import Image
from IPython.display import display

# Filter harmless warnings
import warnings
warnings.filterwarnings("ignore")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Testing data

In [2]:
# df = pd.read_csv('./testFiles/data.csv') 
# df_INPUT_DEPTH = df[['depth_img_I', 'depth_img_II']]
# df_INPUT_RGB = df[['rgb_img_I', 'rgb_img_II']]
# df_OUTPUT = df[['x1','y1','z1','x2','y2','z2']]

In [3]:
ResizeData = transforms.Resize([256, 320], interpolation=transforms.InterpolationMode.NEAREST)

# Functions 
def CreatePointCloud(color_im, depth_im):
    color_raw = o3d.geometry.Image(color_im)
    depth_raw = o3d.geometry.Image(depth_im)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, 1000) # 
    PointCloud = o3d.geometry.PointCloud.create_from_rgbd_image(
      rgbd_image,o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)) # Creates Point Cloud from rgbd image
#     PointCloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down
    return PointCloud

def pick_points(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    numpy_array=np.asarray(pcd.points)
    point_id=vis.get_picked_points()

    return [numpy_array[point_id[0]],numpy_array[point_id[1]]]

def draw_arrow(pcd, points_real, points_extimated):
    lines=[[0,1],[2,3]]
    points = np.concatenate((points_real, points_extimated), axis=0)
    colors = [[1,0,0],[0,1,0]] # Red is REAL and Green is ESTIMATED
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),

    )
    line_set.colors=o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd,line_set])

In [4]:
# RGBimg_begin = load_from_csv(df_INPUT_RGB, 'rgb' ,0)
# DEPTHimg_begin = load_from_csv(df_INPUT_DEPTH, 'depth',0)
# DEPTHimg_end = load_from_csv(df_INPUT_DEPTH, 'depth', 1)

# # Taking first rgb image
# rgb_in = torch.IntTensor(RGBimg_begin.transpose(0,3,1,2))
# gray_in = RGBtoGRAY(rgb_in).div(255)
# # plt.imshow(gray_in[0].numpy().transpose(1,2,0),cmap='gray')

# # Taking depth beginning state of the movement
# depthBeg_in = NormImage(DEPTHimg_begin,5500,reshape=True)

# # Taking depth end state of the movement
# depthEnd_in = NormImage(DEPTHimg_end,5500,reshape=True)

# # Taking depth difference between movements
# depthDiff_in = NormImage(abs(DEPTHimg_begin - DEPTHimg_end),5500,reshape=True)

# # Taking outputs
# axis_out = df_OUTPUT.values
# y_test = torch.Tensor(axis_out)

# X_test = ResizeData(torch.cat((gray_in, depthBeg_in, depthDiff_in),axis=1))
# print(RGBD_input.shape)
# # RGBD_input = DDD_in

In [5]:
from Libraries.dataloader import DataLoader as DL

DATASET_ROOTDIR='/home/el_zlociako/Documents/Praca_inzynierska/Dataset/'
dl = DL()
X_test, y_test = dl.load(DATASET_ROOTDIR, 'files_ArUco/data_ArUco.csv', 'R')

AoRD_testDataset = TensorDataset(X_test, y_test)

In [6]:
test_loader = DataLoader(AoRD_testDataset, batch_size=10, shuffle=True)

In [7]:
# plt.imshow(np.transpose(depthDiff_in[2].numpy(),(1,2,0)))

In [8]:
# plt.imshow(np.transpose(rgb_in[6].numpy(),(1,2,0)))

In [9]:
class AoRNet(nn.Module):
    def __init__(self,pretrained=False ,input_channels=3, output_size=6):
        super().__init__()
        self.resnet50 = models.resnet50(pretrained=pretrained)
        self.resnet50.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet50.fc = nn.Linear(in_features=2048, out_features=output_size, bias=True)
    
    def forward(self, X):
        return self.resnet50(X)

In [10]:
Model = AoRNet()
Model.load_state_dict(torch.load('Modele/ResNet_3IN6OUT_100Epok_Din_Ddiff_L1LOSS_GIT.pt'))
Model.eval()

AoRNet(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [11]:
inv_resize = transforms.Resize(480, interpolation=transforms.InterpolationMode.NEAREST)

In [12]:
# torch.manual_seed(101)
with torch.no_grad():
    for b, (X_validation, y_validation) in enumerate(test_loader):
#         Apply the model
        y_val = Model(X_validation)
#         print(y_val.shape)
        for j in range(y_val.shape[0]):
            X_invNorm = inv_resize(X_validation[j])
            RGB_buff = X_invNorm[0].numpy()*255
#             RGB_buff = np.stack((X_invNorm[0].numpy(),X_invNorm[1].numpy(),X_invNorm[2].numpy()))*255
#             RGB_buff = np.transpose(RGB_buff, (1,2,0))
            RGB_buff = np.ascontiguousarray(RGB_buff, dtype=np.uint8)

            DEPTH_buff = X_invNorm[1].numpy()*5500
            PC = CreatePointCloud(RGB_buff, DEPTH_buff)
            PREDICTED = [[y_val[j][0].cpu().numpy(), y_val[j][1].cpu().numpy(), y_val[j][2].cpu().numpy()],
                         [y_val[j][3].cpu().numpy(), y_val[j][4].cpu().numpy(), y_val[j][5].cpu().numpy()]]
            REAL = [[y_validation[j][0].cpu().numpy(), y_validation[j][1].cpu().numpy(), y_validation[j][2].cpu().numpy()],
                    [y_validation[j][3].cpu().numpy(), y_validation[j][4].cpu().numpy(), y_validation[j][5].cpu().numpy()]]
            draw_arrow(PC, REAL, PREDICTED)

            print(f'--> BATCH: {b+1} <-- | --> ROW: {j} <--')
            print(f'----------------------------------------------------------------------------------------------')
            print(f'{"X1":>12} {"Y1":>12} {"Z1":>12} {"X2":>12} {"Y2":>12} {"Z2":>12}')
            print(f'{"PREDICTED:"}')
            print(f'[[{y_val[j][0]:12.5f}, {y_val[j][1]:12.5f}, {y_val[j][2]:12.5f}], [{y_val[j][3]:12.5f}, {y_val[j][4]:12.5f}, {y_val[j][5]:12.5f}]]')
            print(f'{"REAL:"}')
            print(f'[[{y_validation[j][0]:12.5f}, {y_validation[j][1]:12.5f}, {y_validation[j][2]:12.5f}], [{y_validation[j][3]:12.5f}, {y_validation[j][4]:12.5f}, {y_validation[j][5]:12.5f}]]')
            print(f'{"DIFFERENCE:"}')
            diff = np.abs(y_val.cpu().numpy()-y_validation.cpu().numpy())
            print(f'[[{diff[j][0]:12.5f}, {diff[j][1]:12.5f}, {diff[j][2]:12.5f}], [{diff[j][3]:12.5f}, {diff[j][4]:12.5f}, {diff[j][5]:12.5f}]]')
            print(f'----------------------------------------------------------------------------------------------')
# loss = criterion(y_val, y_validation.cuda())
# diff = np.abs(y_val.cpu().numpy()-y_validation.cpu().numpy())
# print(f'RMSE: {loss:.8f}')


--> BATCH: 1 <-- | --> ROW: 0 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[    -0.17693,     -0.83514,      2.28599], [    -0.18216,      0.97824,      2.40816]]
REAL:
[[     0.02643,     -0.21876,      1.97950], [     0.02001,      0.13011,      1.75150]]
DIFFERENCE:
[[     0.20336,      0.61637,      0.30649], [     0.20217,      0.84813,      0.65666]]
----------------------------------------------------------------------------------------------
--> BATCH: 1 <-- | --> ROW: 1 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[    -0.10783,     -0.53491,      1.63625], [    -0.07558,      0.69583,      1.75612]]
REAL:
[[     0.30275,     -0.59853,      1.29764], [     0.20282,      0.46640,      1.14813]]


--> BATCH: 2 <-- | --> ROW: 4 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[     0.33164,     -0.22615,      1.27174], [     0.36474,      0.49696,      1.39001]]
REAL:
[[     0.15711,      0.19682,      0.52163], [     0.14105,     -0.16195,      0.49663]]
DIFFERENCE:
[[     0.17453,      0.42297,      0.75011], [     0.22369,      0.65891,      0.89338]]
----------------------------------------------------------------------------------------------
--> BATCH: 2 <-- | --> ROW: 5 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[    -0.03126,     -0.57171,      1.80350], [    -0.02764,      0.63630,      1.92760]]
REAL:
[[     0.34555,     -0.51515,      1.32900], [     0.20370,      0.47972,      1.15000]]


--> BATCH: 3 <-- | --> ROW: 8 <--
----------------------------------------------------------------------------------------------
          X1           Y1           Z1           X2           Y2           Z2
PREDICTED:
[[     0.25118,     -0.31342,      1.23134], [     0.26592,      0.55756,      1.43414]]
REAL:
[[    -0.17981,     -0.56752,      1.47500], [    -0.14518,      0.51585,      1.23950]]
DIFFERENCE:
[[     0.43099,      0.25411,      0.24366], [     0.41110,      0.04171,      0.19464]]
----------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
qqqqqqqqqqqqqqq